<a href="https://colab.research.google.com/github/mausombi/projectseekhan/blob/pret/seekhanpretrainedrunningattempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install GPUtil

import torch
import GPUtil
import os
!pip install -q git+https://github.com/huggingface/peft.git
!pip install -q -i https://pypi.org/simple/ bitsandbytes
!pip install -q transformers==4.30
!pip install -q datasets


import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datetime import datetime

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
if 'COLAB_GPU' in os.environ:
    !huggingface-cli login
else:
    notebook_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGr

In [3]:

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "divyabindu77/seekhan_model"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the PEFT model ID
peft_model_id = "divyabindu77/seekhan_model"

# Load the PEFT configuration
config = PeftConfig.from_pretrained(peft_model_id)

# Load the base model with 8-bit precision and automatic device mapping
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    load_in_8bit=True,
    device_map='auto'
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the LoRA (PEFT) model
model = PeftModel.from_pretrained(model, peft_model_id)

# Define your question
question = "What is data abstraction?"

# Format the question into the prompt
eval_prompt = f"{question}\n\n"

# Tokenize the prompt without returning token_type_ids
promptTokenized = tokenizer(
    eval_prompt,
    return_tensors="pt",
    return_token_type_ids=False  # Disable token_type_ids
).to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate the response without token_type_ids
with torch.no_grad():
    output_tokens = model.generate(
        **promptTokenized,
        max_new_tokens=1024
    )
    generated_text = tokenizer.decode(
        output_tokens[0],
        skip_special_tokens=True
    )
    print(generated_text)

# Clear CUDA cache
torch.cuda.empty_cache()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

What is data abstraction?

Data abstraction is a technique used in computer programming to separate the implementation details of a data type from its interface, allowing the implementation to be changed without affecting the code that uses it. This is often achieved through the use of abstract data types (ADTs), which are defined by the operations they support rather than their specific implementation, or through the use of interfaces and classes in object-oriented programming languages.

Data abstraction helps to reduce the complexity of software systems by allowing code to be written in a modular and flexible way, and by hiding the underlying details of data types from the user. It is a fundamental concept in software design and is used in many areas of programming, including database design, networking, and user interface design.

There are several benefits of using data abstraction, including:

1. Code flexibility: Data abstraction allows code to be written in a way that is indepe

In [5]:
# Define your question
question = "Data abstraction"

# Format the question into the prompt
eval_prompt = f"The topic is this- {question}, You are supposed to generate Multiple choice questions on it.\n\n"

# Tokenize the prompt without returning token_type_ids
promptTokenized = tokenizer(
    eval_prompt,
    return_tensors="pt",
    return_token_type_ids=False  # Disable token_type_ids
).to("cuda")

# Set the model to evaluation mode
model.eval()

# Generate the response without token_type_ids
with torch.no_grad():
    output_tokens = model.generate(
        **promptTokenized,
        max_new_tokens=1024
    )
    generated_text = tokenizer.decode(
        output_tokens[0],
        skip_special_tokens=True
    )
    print(generated_text)

# Clear CUDA cache
torch.cuda.empty_cache()


The topic is this- Data abstraction, You are supposed to generate Multiple choice questions on it.

Data abstraction is a technique used in computer programming to separate the implementation details of a data type from its interface, allowing the implementation to be changed without affecting the code that uses it.

Here are some multiple choice questions on data abstraction:

1. What is data abstraction?
a) A technique for hiding the implementation details of data types
b) A technique for exposing the implementation details of data types
c) A technique for creating complex data types
d) A technique for simplifying code

Answer: a) A technique for hiding the implementation details of data types

2. What is the purpose of data abstraction?
a) To make code more complex
b) To make code easier to understand
c) To hide the implementation details of data types
d) To expose the implementation details of data types

Answer: c) To hide the implementation details of data types

3. How does data

In [6]:
import torch
torch.cuda.empty_cache()


### ENTER YOUR QUESTION BELOW

question = "What is data abstraction?"

# Format the question
eval_prompt = f"{question}\n\n"

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**promptTokenized, max_new_tokens = 1024)[0], skip_special_tokens=True))
torch.cuda.empty_cache()